In [4]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "close_price": "float",
    "highest_price": "float",
    "lowest_price": "float", 
    "volumn": "int"}
for dow_name in dow_files_names:
    print(dow_name)
    df = pd.read_csv(dow_files_addr + dow_name, sep=",", names=headers,dtype=dtypes, header=None)
    dow_dfs.append(df)
for etf_name in etf_files_names:
    print(etf_name)
    df = pd.read_csv(etf_files_addr + etf_name, sep=",", names=headers,dtype=dtypes, header=None)
    etf_dfs.append(df)

AA.txt
AAPL.txt
AXP.txt
BA.txt
BAC.txt
C.txt
CAT.txt
CSCO.txt
CVX.txt
DD.txt
DIS.txt
DOW.txt
DWDP.txt
GE.txt
GM.txt
GS.txt
HD.txt
HON.txt
HPQ.txt
IBM.txt
INTC.txt
JNJ.txt
JPM.txt
KFT.txt
KO.txt
MCD.txt
MDLZ.txt
MMM.txt
MO.txt
MRK.txt
MSFT.txt
NKE.txt
PFE.txt
PG.txt
PGP.txt
RTX.txt
T.txt
TRV.txt
UNH.txt
UTX.txt
V.txt
VZ.txt
WBA.txt
WMT.txt
XOM.txt
DIA.txt
EEM.txt
EFA.txt
ERX.txt
EWZ.txt
FAS.txt
FAZ.txt
FXE.txt
FXI.txt
GDX.txt
GLD.txt
HYG.txt
IVV.txt
IWF.txt
IWM.txt
IWO.txt
IYR.txt
JNK.txt
LQD.txt
MDY.txt
OIH.txt
QLD.txt
QQQ.txt
RSX.txt
SDS.txt
SH.txt
SHY.txt
SLV.txt
SPY.txt
SSO.txt
TLT.txt
TNA.txt
TZA.txt
UPRO.txt
USO.txt
VTI.txt
VWO.txt
VXX.txt
XLB.txt
XLE.txt
XLF.txt
XLI.txt
XLK.txt
XLP.txt
XLU.txt
XLV.txt
XLY.txt
XME.txt
XOP.txt
XRT.txt


In [3]:
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

dow_dfs = []
etf_dfs = []
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "close_price": "float",
    "highest_price": "float",
    "lowest_price": "float", 
    "volumn": "int"}
for dow_name in dow_files_names:
    df = pd.read_csv(dow_files_addr + dow_name, sep=",", header=None)
    dow_dfs.append(df)
for etf_name in etf_files_names:
    df = pd.read_csv(etf_files_addr + etf_name, sep=",", header=None)
    etf_dfs.append(df)

In [73]:
 dtypes={"date": "str", 
        "time":"str",
        "open_price": "float", 
        "close_price": "float",
        "highest_price": "float",
        "lowest_price": "float", 
        "volumn": "int"}

In [74]:
headers = ["date", "time", "open_price", "close_price", "highest_price", "lowest_price", "volumn"]

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

In [ ]:
df = pd.read_csv("./Data/Dow_30_1_min/AA.txt", sep=",",names=headers,dtype=dtypes, parse_dates=['date'], date_parser=dateparse, header=None)


In [ ]:
df

In [ ]:
df.dtypes